In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("IcebergMinIO")
    .config(
        "spark.jars.packages",
        "org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.6.1,"
        "org.apache.hadoop:hadoop-aws:3.3.4"
    )
    .config("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.demo.type", "hadoop")
    .config("spark.sql.catalog.demo.warehouse", "s3a://bronze/warehouse/")
    .config("spark.hadoop.fs.s3a.access.key", "teste")
    .config("spark.hadoop.fs.s3a.secret.key", "password")
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
25/10/19 20:54:24 WARN Utils: Your hostname, DESKTOP-4LPUPP7 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/10/19 20:54:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/andre/miniforge3/envs/iceberg/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/andre/.ivy2/cache
The jars for the packages stored in: /home/andre/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.4_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-79449eac-4ec4-48b4-af40-a413fcfc832b;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.6.1 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 192ms :: artifacts dl 8ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.6.1 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	-------------------

25/10/19 20:54:42 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
spark.sql("CREATE DATABASE IF NOT EXISTS demo.db_teste")


DataFrame[]

In [5]:
spark.sql("""
    CREATE TABLE demo.db_teste.teste (
        id INT,
        nome STRING,
        valor DOUBLE
    ) USING ICEBERG
""")

print("✅ Tabela criada com sucesso!")

✅ Tabela criada com sucesso!


In [6]:
spark.sql("INSERT INTO demo.db_teste.teste VALUES (1, 'A', 10.5), (2, 'B', 20.0)")

DataFrame[]

In [7]:
spark.sql("SELECT * FROM demo.db_teste.teste").show()

+---+----+-----+
| id|nome|valor|
+---+----+-----+
|  1|   A| 10.5|
|  2|   B| 20.0|
+---+----+-----+



In [5]:
spark.sql("INSERT INTO demo.db_teste_iceberg.teste VALUES (3, 'C', 11.5), (4, 'B', 21.0)")

DataFrame[]

In [6]:
spark.sql("SELECT * FROM demo.db_teste_iceberg.teste").show()

+---+----+-----+
| id|nome|valor|
+---+----+-----+
|  1|   A| 10.5|
|  3|   C| 11.5|
|  2|   B| 20.0|
|  4|   B| 21.0|
+---+----+-----+



In [8]:
spark.sql("show catalogs").show()

+-------------+
|      catalog|
+-------------+
|         demo|
|spark_catalog|
+-------------+



In [9]:
spark.sql("SHOW DATABASES IN demo").show()




+----------------+
|       namespace|
+----------------+
|db_teste_iceberg|
+----------------+



In [10]:
spark.sql("SHOW TABLES IN demo.db_teste_iceberg").show()


+----------------+---------+-----------+
|       namespace|tableName|isTemporary|
+----------------+---------+-----------+
|db_teste_iceberg|    teste|      false|
+----------------+---------+-----------+



In [13]:
spark.sql("SELECT * FROM demo.db_teste_iceberg.teste.history").show(truncate=False)


+-----------------------+-------------------+-------------------+-------------------+
|made_current_at        |snapshot_id        |parent_id          |is_current_ancestor|
+-----------------------+-------------------+-------------------+-------------------+
|2025-10-08 20:18:02.407|3295861635440732690|NULL               |true               |
|2025-10-08 20:18:05.447|8523423454150741093|3295861635440732690|true               |
+-----------------------+-------------------+-------------------+-------------------+



In [14]:
spark.sql("SELECT * FROM demo.db_teste_iceberg.teste.files").show(truncate=False)


+-------+---------------------------------------------------------------------------------------------------------------+-----------+-------+------------+------------------+---------------------------+------------------------+------------------------+----------------+---------------------------------------------------------------+---------------------------------------------------------------+------------+-------------+------------+-------------+---------------------------------------------------------------------------+
|content|file_path                                                                                                      |file_format|spec_id|record_count|file_size_in_bytes|column_sizes               |value_counts            |null_value_counts       |nan_value_counts|lower_bounds                                                   |upper_bounds                                                   |key_metadata|split_offsets|equality_ids|sort_order_id|readable_metrics       